In [207]:
import warnings
warnings.filterwarnings("ignore")
# imports
from math import sqrt, log, inf
import gym
import copy
import numpy as np
from gym_go.gogame import turn, valid_moves

In [208]:
# Global constants
UCB_C = 2

BLACK = 0
WHITE = 1
TURN_CHNL = 2
INVD_CHNL = 3
PASS_CHNL = 4
DONE_CHNL = 5

# Monte Carlo Three Search

1. Selection
    - Taverse the tree to find greatest UCB-score
2. Expansion
    - If the selected leaf node has been visited before expand by adding weighted game action
3. Rollout
    - Simulate the game until end-condition from the expanded leaf
4. Back-propagation
    - Updating the value of each ancestor node of the expanded leaf


In [209]:
def get_legal_move(env):
    board_shape = env.state().shape[1:]
    pass_id = np.prod(board_shape)
    action = env.action_space.sample() # pick random action
    action2d = action // board_shape[0], action % board_shape[1], action
    while action2d[2] != pass_id and env.state()[3, action2d[0], action2d[1]] == 1:
        action = env.action_space.sample() # pick random action
        action2d = action // board_shape[0], action % board_shape[1], action
    return action2d[2]

def equals(state_a, state_b):
    for i in range(0, 6):
        if not np.array_equal(state_a[i], state_b[i]):
            return False
    return True


In [210]:
class Node():
    def __init__(self, env, parent, action):
        self.env : gym.Env = env # This env will be altered by the other player
        self.value : int = 0 # Value estimate
        self.trials : int = 0 # Number of trials for this node
        self.parent : Node = parent # Parent node of this node
        self.children : list[Node] = [] # List of children of this node
        self.action : int = action # The step action made by this node
    
    # calculate a Upper Confidence Bound
    def ucb(self, total_trials):
        value = self.value
        if turn(self.env.state()) == BLACK:
            value = -self.value
        return value + ( UCB_C * sqrt(log(total_trials) / self.trials) )
    
    # Add a new node to a leaf node
    def expansion(self):
        if self.env.done:
            return
        board_shape = self.env.state().shape[1:]
        pass_id = np.prod(board_shape)
        for action in range(0, pass_id + 1):
            action2d = action // board_shape[0], action % board_shape[1]
            if action == pass_id or self.env.state()[3, action2d[0], action2d[1]] == 0:
                child_env = copy.deepcopy(self.env)
                child_env.step(action)
                self.children.append(Node(child_env, self, action))

    # Simulate game from current move until end-condition returning the score
    def rollout(self):
        if self.env.done:
            return self.env.reward()
        
        rollout_env = copy.deepcopy(self.env)
        rollout_result = 0
        done = False
        while not done:
            random_action = get_legal_move(rollout_env)
            _, reward, done, _ = rollout_env.step(random_action)
            rollout_result += reward
        return rollout_result

In [211]:
class Monte_Carlo_Tree_Search():
    def __init__(self, size, komi):
        self.env : gym.Env = gym.make('gym_go:go-v0', size=size, komi=komi, reward_method='heuristic')
        self.env.reset()
        self.number_of_trials : int = 0
        self.root = Node(self.env, None, None)
    
    # Update scores of all parent nodes after rollout
    def back_propagation(self, rollout_node: Node, rollout_result):
        current_node = rollout_node
        while current_node != None:
            current_node.trials += 1
            current_node.value += rollout_result
            current_node = current_node.parent
        self.number_of_trials += 1
    
    # find and return the leaf node with the highest UCB-score 
    def selection(self):
        selected_child = self.root
        current_node = self.root
        while len(current_node.children) > 0:
            selected_child = current_node.children[0]
            current_best_ucb = -inf

            for child in current_node.children:
                if child.trials == 0:
                    return child

                child_ucb = child.ucb(self.number_of_trials)

                if child_ucb > current_best_ucb:
                    selected_child = child
                    current_best_ucb = child_ucb

            current_node = selected_child

        return selected_child
        
    def run(self, iterations):
        selected_node = self.root
        selected_node.expansion()
        selected_node = self.root.children[0]

        run = 0
        while run < iterations:
            selected_node = self.selection()
            # print("Run:", run, ": Selection", selected_node.action)

            if selected_node.env.done:
                self.back_propagation(selected_node, selected_node.env.reward())
                run += 1
                continue

            if selected_node.trials > 0:
                # print("Run:", run, ": Expansion:")
                selected_node.expansion()
                selected_node = selected_node.children[0]

            rollout_result = selected_node.rollout()
            # print("Run:", run, ": Rollout", rollout_result)
            
            self.back_propagation(selected_node, rollout_result)
            run += 1
    
    def find_node_from_state(self, node: Node, state):
        if equals(node.env.state(), state):
            return node

        for child in node.children:
            if equals(child.env.state(), state):
                return child
            
            res = self.find_node_from_state(child, state)
            if res != None and equals(res.env.state(), state):
                return res

    def get_move_from_env(self, env):
        node = self.find_node_from_state(self.root, env.state())
        
        if node != None:
            if len(node.children) == 0:
                node.expansion()
            
            best_child = None
            current_best_value = -inf
            for child in node.children:
                value = child.value
                if turn(self.env.state()) == BLACK:
                    value = -child.value

                if value > current_best_value:
                    best_child = child
                    current_best_value = value
                    
            if best_child != None:
                if len(best_child.children) == 0:
                    best_child.expansion()
                return best_child
                
        return None

In [212]:
model = Monte_Carlo_Tree_Search(5,0)
model.run(1000)

In [213]:
def play_game(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

    if node != None:
        model.back_propagation(node, go_env.reward())

def play_game_no_render(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)

    if node != None:
        model.back_propagation(node, go_env.reward())
    
    return go_env

def play_model_vs_model(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

    if node1 != None:
        model1.back_propagation(node1, go_env.reward())
    if node2 != None:
        model2.back_propagation(node2, go_env.reward())

def play_model_vs_model_no_render(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, reward = go_env.step(action)

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, reward = go_env.step(action)

    return go_env

In [214]:
play_game(get_legal_move, model, copy.deepcopy(model.env))

	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

	0 1 2 3 4 
0	●═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 1, White Area: 1

	0 1 2 3 4 
0	●═╤═╤═╤═╗
1	╟─○─┼─┼─╢
2	╟─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 1

	0 1 2 3 4 
0	●═●═╤═╤═╗
1	╟─○─┼─┼─╢
2	╟─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 2

	0 1 2 3 4 
0	●═●═╤═╤═╗
1	╟─○─┼─┼─╢
2	○─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 3, White Area: 2

	0 1 2 3 4 
0	●═●═●═╤═╗
1	╟─○─┼─┼─╢
2	○─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 3, White Area: 3

	0 1 2 3 4 
0	●═●═●═╤═╗
1	╟─○─┼─○─╢
2	○─┼─┼─┼─╢
3	○

In [215]:
model2 = Monte_Carlo_Tree_Search(5,0)
model2.run(100)

In [ ]:
play_model_vs_model(model, model2, copy.deepcopy(model.env))

	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─○
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

	0 1 2 3 4 
0	●═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─○
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 1, White Area: 1

	0 1 2 3 4 
0	●═╤═╤═╤═╗
1	╟─┼─┼─┼─○
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─○
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 1

	0 1 2 3 4 
0	●═●═╤═╤═╗
1	╟─┼─┼─┼─○
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─○
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 2

	0 1 2 3 4 
0	●═●═╤═╤═╗
1	╟─┼─┼─┼─○
2	╟─┼─┼─○─╢
3	╟─┼─┼─┼─○
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 4, White Area: 2

	0 1 2 3 4 
0	●═●═●═╤═╗
1	╟─┼─┼─┼─○
2	╟─┼─┼─○─╢
3	╟─┼─┼─┼─○
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 4, White Area: 3

	0 1 2 3 4 
0	●═●═●═╤═╗
1	╟─┼─┼─┼─○
2	╟─┼─┼─○─╢
3	╟

In [ ]:
for i in range(1,10):
    env = play_game_no_render(get_legal_move, model, copy.deepcopy(model.env))
    if env.reward() < 0:
        print("White won!")
    if env.reward() > 0:
        print("Black won!")

Black won!
White won!
Black won!
Black won!
White won!
White won!
Black won!
White won!
Black won!
